In [1]:
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, LeakyReLU, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.engine.input_layer import Input
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image

Using TensorFlow backend.


In [2]:
def preprocess_input(im):
    return (im.astype(np.float32) - 127.5)/127.5

def reformat_image(im):
    return ((im * 127.5) + 127.5).astype(np.int32)

In [3]:
# This class exists as to keep the entire array of
# images out of RAM and instead load them in as needed
class ImageLoader():
    def __init__(self, files, func):
        self.files = files
        self.c_index = 0
        self.max = len(self.files)
        self.func = func
    
    def getNextFiles(self, num):
        arr = []
        for i in range(num):
            arr.append(self.load_image(self.files[self.c_index]))
            self.c_index += 1
            if self.c_index == self.max:
                print("Looping data now")
                self.c_index = 0
        return self.func(np.array(arr))
        
    def load_image(self, filename):
        img = Image.open(filename)
        img = img.convert('1')
        img = img.resize((28,28))
        img = list(img.getdata())
        img = np.array(img)
        return img.reshape((28,28,1))

In [6]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        #X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=3)

        # Create the image loader and the give it the collection of images
        image_locs = glob("fruits/fruits-360/Training/*/*.jpg")
        il = ImageLoader(image_locs, preprocess_input)
        
        half_batch = int(batch_size / 2)

        print("Starting the training...")
        
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            imgs = il.getNextFiles(half_batch)

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/epoch_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=1000, batch_size=32, save_interval=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

95 [D loss: 0.019822, acc.: 100.00%] [G loss: 4.413434]
96 [D loss: 0.022950, acc.: 100.00%] [G loss: 4.213750]
97 [D loss: 0.040933, acc.: 100.00%] [G loss: 4.249125]
98 [D loss: 0.026635, acc.: 100.00%] [G loss: 4.232487]
99 [D loss: 0.035921, acc.: 100.00%] [G loss: 4.077765]
100 [D loss: 0.023977, acc.: 100.00%] [G loss: 4.082221]
101 [D loss: 0.019261, acc.: 100.00%] [G loss: 4.057758]
102 [D loss: 0.022960, acc.: 100.00%] [G loss: 4.099289]
103 [D loss: 0.024623, acc.: 100.00%] [G loss: 4.095257]
104 [D loss: 0.072561, acc.: 100.00%] [G loss: 3.938179]
105 [D loss: 0.091011, acc.: 96.88%] [G loss: 3.696545]
106 [D loss: 0.058303, acc.: 100.00%] [G loss: 3.963068]
107 [D loss: 0.031568, acc.: 100.00%] [G loss: 4.189562]
108 [D loss: 0.018384, acc.: 100.00%] [G loss: 4.143913]
109 [D loss: 0.018077, acc.: 100.00%] [G loss: 4.334985]
110 [D loss: 0.029703, acc.: 100.00%] [G loss: 4.277511]
111 [D loss: 0.024660, acc.: 100.00%] [G loss: 4.332917]
112 [D loss: 0.030020, acc.: 100.00%]

241 [D loss: 0.341634, acc.: 78.12%] [G loss: 2.233799]
242 [D loss: 0.529557, acc.: 75.00%] [G loss: 2.208550]
243 [D loss: 0.351464, acc.: 81.25%] [G loss: 2.232465]
244 [D loss: 0.914634, acc.: 50.00%] [G loss: 1.568069]
245 [D loss: 0.399530, acc.: 78.12%] [G loss: 1.886619]
246 [D loss: 0.601416, acc.: 68.75%] [G loss: 2.214753]
247 [D loss: 0.665363, acc.: 62.50%] [G loss: 1.663875]
248 [D loss: 0.608531, acc.: 59.38%] [G loss: 1.420885]
249 [D loss: 0.492425, acc.: 75.00%] [G loss: 2.299768]
250 [D loss: 0.497928, acc.: 75.00%] [G loss: 2.100084]
251 [D loss: 0.386740, acc.: 81.25%] [G loss: 2.579005]
252 [D loss: 0.261028, acc.: 90.62%] [G loss: 3.200799]
253 [D loss: 0.550462, acc.: 71.88%] [G loss: 2.165175]
254 [D loss: 0.257891, acc.: 100.00%] [G loss: 2.023634]
255 [D loss: 0.734972, acc.: 59.38%] [G loss: 1.574628]
256 [D loss: 0.320138, acc.: 87.50%] [G loss: 2.146706]
257 [D loss: 0.265724, acc.: 96.88%] [G loss: 2.551865]
258 [D loss: 0.378605, acc.: 87.50%] [G loss: 2

389 [D loss: 0.106295, acc.: 100.00%] [G loss: 1.905347]
390 [D loss: 0.128668, acc.: 96.88%] [G loss: 1.929262]
391 [D loss: 0.135832, acc.: 100.00%] [G loss: 1.870837]
392 [D loss: 0.165949, acc.: 100.00%] [G loss: 1.990878]
393 [D loss: 0.188089, acc.: 100.00%] [G loss: 2.046349]
394 [D loss: 0.246687, acc.: 93.75%] [G loss: 2.076720]
395 [D loss: 0.264172, acc.: 93.75%] [G loss: 1.950471]
396 [D loss: 0.179824, acc.: 96.88%] [G loss: 2.029299]
397 [D loss: 0.258700, acc.: 90.62%] [G loss: 2.032174]
398 [D loss: 0.280641, acc.: 87.50%] [G loss: 2.025829]
399 [D loss: 0.240887, acc.: 93.75%] [G loss: 1.848790]
400 [D loss: 0.228461, acc.: 93.75%] [G loss: 2.171149]
401 [D loss: 1.886514, acc.: 40.62%] [G loss: 1.413358]
402 [D loss: 0.734008, acc.: 56.25%] [G loss: 1.395943]
403 [D loss: 0.410914, acc.: 87.50%] [G loss: 1.542068]
404 [D loss: 0.449711, acc.: 87.50%] [G loss: 1.835602]
405 [D loss: 0.389539, acc.: 84.38%] [G loss: 1.672044]
406 [D loss: 0.591582, acc.: 68.75%] [G loss

535 [D loss: 0.315503, acc.: 93.75%] [G loss: 2.540625]
536 [D loss: 0.780695, acc.: 65.62%] [G loss: 2.144283]
537 [D loss: 0.142256, acc.: 96.88%] [G loss: 2.068719]
538 [D loss: 0.115480, acc.: 96.88%] [G loss: 2.389652]
539 [D loss: 0.071945, acc.: 100.00%] [G loss: 2.888242]
540 [D loss: 0.075806, acc.: 100.00%] [G loss: 3.017581]
541 [D loss: 0.062754, acc.: 100.00%] [G loss: 2.961560]
542 [D loss: 0.406714, acc.: 81.25%] [G loss: 2.476636]
543 [D loss: 0.103036, acc.: 100.00%] [G loss: 2.384413]
544 [D loss: 0.086164, acc.: 100.00%] [G loss: 2.760684]
545 [D loss: 0.050666, acc.: 100.00%] [G loss: 2.722765]
546 [D loss: 0.082556, acc.: 100.00%] [G loss: 2.628155]
547 [D loss: 0.264212, acc.: 96.88%] [G loss: 2.834663]
548 [D loss: 3.784229, acc.: 40.62%] [G loss: 1.445485]
549 [D loss: 0.343421, acc.: 75.00%] [G loss: 1.665739]
550 [D loss: 0.161429, acc.: 93.75%] [G loss: 2.952101]
551 [D loss: 0.062246, acc.: 100.00%] [G loss: 3.032698]
552 [D loss: 0.056651, acc.: 100.00%] [G

681 [D loss: 0.219062, acc.: 93.75%] [G loss: 2.674374]
682 [D loss: 0.114085, acc.: 96.88%] [G loss: 2.801945]
683 [D loss: 0.068589, acc.: 100.00%] [G loss: 2.693017]
684 [D loss: 0.323606, acc.: 81.25%] [G loss: 2.713141]
685 [D loss: 0.161373, acc.: 93.75%] [G loss: 2.697949]
686 [D loss: 0.269417, acc.: 90.62%] [G loss: 2.680665]
687 [D loss: 0.157800, acc.: 96.88%] [G loss: 2.540521]
688 [D loss: 0.205971, acc.: 90.62%] [G loss: 2.526914]
689 [D loss: 0.263069, acc.: 90.62%] [G loss: 2.256675]
690 [D loss: 0.181853, acc.: 90.62%] [G loss: 2.877140]
691 [D loss: 0.099141, acc.: 100.00%] [G loss: 2.795810]
692 [D loss: 0.260851, acc.: 93.75%] [G loss: 2.804490]
693 [D loss: 0.417223, acc.: 81.25%] [G loss: 2.376296]
694 [D loss: 0.298504, acc.: 90.62%] [G loss: 2.472589]
695 [D loss: 0.507728, acc.: 81.25%] [G loss: 2.444933]
696 [D loss: 0.697326, acc.: 65.62%] [G loss: 3.223728]
697 [D loss: 0.353810, acc.: 90.62%] [G loss: 2.906591]
698 [D loss: 0.133166, acc.: 100.00%] [G loss:

827 [D loss: 0.359451, acc.: 100.00%] [G loss: 1.801857]
828 [D loss: 0.257666, acc.: 100.00%] [G loss: 1.924853]
829 [D loss: 0.346026, acc.: 96.88%] [G loss: 1.885563]
830 [D loss: 0.299279, acc.: 96.88%] [G loss: 1.754140]
831 [D loss: 0.416613, acc.: 93.75%] [G loss: 1.549296]
832 [D loss: 0.317040, acc.: 100.00%] [G loss: 1.667851]
833 [D loss: 0.290346, acc.: 100.00%] [G loss: 1.942729]
834 [D loss: 0.234375, acc.: 100.00%] [G loss: 1.847360]
835 [D loss: 0.379512, acc.: 90.62%] [G loss: 1.564605]
836 [D loss: 0.453358, acc.: 90.62%] [G loss: 1.546203]
837 [D loss: 0.462372, acc.: 84.38%] [G loss: 1.307040]
838 [D loss: 0.583972, acc.: 71.88%] [G loss: 1.347588]
839 [D loss: 0.689767, acc.: 53.12%] [G loss: 1.351856]
840 [D loss: 0.847183, acc.: 43.75%] [G loss: 1.126288]
841 [D loss: 0.657433, acc.: 59.38%] [G loss: 1.123682]
842 [D loss: 0.885683, acc.: 37.50%] [G loss: 1.140212]
843 [D loss: 0.698556, acc.: 53.12%] [G loss: 1.222360]
844 [D loss: 0.534689, acc.: 78.12%] [G los

973 [D loss: 0.938391, acc.: 75.00%] [G loss: 2.128492]
974 [D loss: 1.605074, acc.: 43.75%] [G loss: 1.701940]
975 [D loss: 1.070082, acc.: 43.75%] [G loss: 1.604836]
976 [D loss: 0.622155, acc.: 65.62%] [G loss: 1.939831]
977 [D loss: 0.470843, acc.: 78.12%] [G loss: 1.946880]
978 [D loss: 0.296591, acc.: 100.00%] [G loss: 2.380187]
979 [D loss: 0.207811, acc.: 100.00%] [G loss: 2.556520]
980 [D loss: 0.193862, acc.: 100.00%] [G loss: 2.563452]
981 [D loss: 0.275668, acc.: 96.88%] [G loss: 2.343353]
982 [D loss: 0.290817, acc.: 93.75%] [G loss: 2.033949]
983 [D loss: 0.205270, acc.: 100.00%] [G loss: 2.405761]
984 [D loss: 0.280764, acc.: 100.00%] [G loss: 2.474721]
985 [D loss: 0.206580, acc.: 100.00%] [G loss: 2.322716]
986 [D loss: 0.268857, acc.: 100.00%] [G loss: 2.216783]
987 [D loss: 0.211902, acc.: 100.00%] [G loss: 2.189634]
988 [D loss: 0.310379, acc.: 90.62%] [G loss: 2.078443]
989 [D loss: 0.321788, acc.: 90.62%] [G loss: 1.883514]
990 [D loss: 0.293346, acc.: 96.88%] [G 

In [7]:
gan

In [ ]:
gan.train(epochs=10000, batch_size=256, save_interval=10)

Starting the training...
0 [D loss: 3.705827, acc.: 42.58%] [G loss: 2.328307]
1 [D loss: 0.758030, acc.: 63.67%] [G loss: 1.900742]
2 [D loss: 0.259875, acc.: 89.45%] [G loss: 2.271671]
3 [D loss: 0.179699, acc.: 94.92%] [G loss: 2.554211]
4 [D loss: 0.168700, acc.: 99.61%] [G loss: 2.592947]
5 [D loss: 0.376902, acc.: 86.72%] [G loss: 2.441292]
6 [D loss: 0.130788, acc.: 99.22%] [G loss: 2.626394]
7 [D loss: 1.298695, acc.: 56.25%] [G loss: 1.815416]
8 [D loss: 1.341855, acc.: 28.12%] [G loss: 1.444080]
9 [D loss: 0.515444, acc.: 72.66%] [G loss: 2.073139]
10 [D loss: 0.177885, acc.: 98.44%] [G loss: 2.473165]
11 [D loss: 0.119940, acc.: 99.61%] [G loss: 2.496236]
12 [D loss: 0.108357, acc.: 100.00%] [G loss: 2.534445]
13 [D loss: 0.222240, acc.: 96.48%] [G loss: 2.333335]
14 [D loss: 0.285660, acc.: 90.23%] [G loss: 2.302296]
15 [D loss: 0.118477, acc.: 99.22%] [G loss: 2.465120]
16 [D loss: 0.224882, acc.: 93.36%] [G loss: 2.567767]
17 [D loss: 0.510965, acc.: 76.17%] [G loss: 2.03

148 [D loss: 1.002682, acc.: 44.14%] [G loss: 1.313665]
149 [D loss: 0.846364, acc.: 55.86%] [G loss: 1.311158]
150 [D loss: 0.441653, acc.: 80.86%] [G loss: 1.694658]
151 [D loss: 0.252556, acc.: 94.53%] [G loss: 1.984070]
152 [D loss: 0.216487, acc.: 95.70%] [G loss: 2.089152]
153 [D loss: 0.136691, acc.: 99.61%] [G loss: 2.135785]
154 [D loss: 0.101053, acc.: 100.00%] [G loss: 2.240618]
155 [D loss: 0.108324, acc.: 100.00%] [G loss: 2.290051]
156 [D loss: 0.111345, acc.: 99.61%] [G loss: 2.291627]
157 [D loss: 0.463494, acc.: 80.86%] [G loss: 1.852417]
158 [D loss: 0.407193, acc.: 89.06%] [G loss: 1.747687]
159 [D loss: 0.243652, acc.: 98.05%] [G loss: 1.989084]
160 [D loss: 0.216233, acc.: 98.44%] [G loss: 2.090848]
161 [D loss: 0.182224, acc.: 98.44%] [G loss: 2.177249]
162 [D loss: 0.201718, acc.: 98.83%] [G loss: 2.242907]
163 [D loss: 0.332605, acc.: 88.67%] [G loss: 2.002111]
164 [D loss: 0.334025, acc.: 91.02%] [G loss: 1.896206]
165 [D loss: 0.286469, acc.: 93.36%] [G loss: 

295 [D loss: 2.556230, acc.: 39.84%] [G loss: 1.236752]
296 [D loss: 0.603543, acc.: 72.27%] [G loss: 1.678656]
297 [D loss: 0.333583, acc.: 92.19%] [G loss: 1.838009]
298 [D loss: 0.297466, acc.: 89.45%] [G loss: 1.916323]
299 [D loss: 0.245161, acc.: 94.92%] [G loss: 1.954282]
300 [D loss: 0.123923, acc.: 99.22%] [G loss: 2.163203]
301 [D loss: 0.103178, acc.: 99.61%] [G loss: 2.358757]
302 [D loss: 0.298661, acc.: 91.41%] [G loss: 2.236749]
303 [D loss: 0.700398, acc.: 70.70%] [G loss: 1.674239]
304 [D loss: 0.442325, acc.: 80.47%] [G loss: 1.778587]
305 [D loss: 0.158227, acc.: 99.22%] [G loss: 2.139990]
306 [D loss: 0.076713, acc.: 100.00%] [G loss: 2.407315]
307 [D loss: 0.420387, acc.: 83.59%] [G loss: 2.080340]
308 [D loss: 0.439349, acc.: 83.98%] [G loss: 1.904501]
309 [D loss: 0.122241, acc.: 99.61%] [G loss: 2.278681]
310 [D loss: 0.092700, acc.: 99.61%] [G loss: 2.381839]
311 [D loss: 0.092103, acc.: 99.61%] [G loss: 2.518366]
312 [D loss: 0.145941, acc.: 98.05%] [G loss: 2

443 [D loss: 0.226173, acc.: 96.09%] [G loss: 2.455596]
444 [D loss: 0.854864, acc.: 53.12%] [G loss: 2.022527]
445 [D loss: 0.561512, acc.: 70.31%] [G loss: 2.323994]
446 [D loss: 0.410771, acc.: 82.42%] [G loss: 2.271793]
Looping data now
447 [D loss: 0.977987, acc.: 57.03%] [G loss: 1.921733]
448 [D loss: 0.711177, acc.: 61.33%] [G loss: 2.124262]
449 [D loss: 0.153030, acc.: 100.00%] [G loss: 2.637516]
450 [D loss: 0.245153, acc.: 93.36%] [G loss: 2.298789]
451 [D loss: 0.788109, acc.: 63.28%] [G loss: 1.587879]
452 [D loss: 0.546049, acc.: 70.70%] [G loss: 1.803159]
453 [D loss: 0.236011, acc.: 93.36%] [G loss: 2.199493]
454 [D loss: 0.274781, acc.: 95.31%] [G loss: 2.043816]
455 [D loss: 0.383348, acc.: 89.84%] [G loss: 1.971980]
456 [D loss: 0.257224, acc.: 95.31%] [G loss: 2.070576]
457 [D loss: 0.184258, acc.: 99.22%] [G loss: 2.337701]
458 [D loss: 0.115869, acc.: 99.61%] [G loss: 2.376185]
459 [D loss: 0.098851, acc.: 100.00%] [G loss: 2.439504]
460 [D loss: 0.174230, acc.: 

589 [D loss: 0.259897, acc.: 92.19%] [G loss: 1.971206]
590 [D loss: 0.187309, acc.: 98.05%] [G loss: 1.838586]
591 [D loss: 0.158047, acc.: 97.66%] [G loss: 1.893499]
592 [D loss: 0.195383, acc.: 94.92%] [G loss: 1.832195]
593 [D loss: 0.889955, acc.: 46.09%] [G loss: 1.336815]
594 [D loss: 0.446970, acc.: 78.91%] [G loss: 1.420593]
595 [D loss: 0.920316, acc.: 51.17%] [G loss: 1.290673]
596 [D loss: 0.691083, acc.: 67.58%] [G loss: 1.359375]
597 [D loss: 1.134138, acc.: 39.84%] [G loss: 1.305274]
598 [D loss: 0.511888, acc.: 76.56%] [G loss: 1.510220]
599 [D loss: 0.402533, acc.: 87.89%] [G loss: 1.706320]
600 [D loss: 0.357772, acc.: 86.33%] [G loss: 1.870040]
601 [D loss: 0.197658, acc.: 96.09%] [G loss: 2.026959]
602 [D loss: 0.200595, acc.: 96.09%] [G loss: 2.103725]
603 [D loss: 0.151622, acc.: 97.66%] [G loss: 2.228704]
604 [D loss: 0.542859, acc.: 71.88%] [G loss: 1.891513]
605 [D loss: 0.655304, acc.: 65.62%] [G loss: 1.530430]
606 [D loss: 0.469934, acc.: 79.69%] [G loss: 1.

...

2907 [D loss: 0.464137, acc.: 75.00%] [G loss: 1.524394]
2908 [D loss: 0.422661, acc.: 83.98%] [G loss: 1.775902]
2909 [D loss: 0.232575, acc.: 96.48%] [G loss: 1.878963]
2910 [D loss: 0.208295, acc.: 96.88%] [G loss: 1.993071]
2911 [D loss: 0.648873, acc.: 89.84%] [G loss: 1.810808]
2912 [D loss: 2.978370, acc.: 29.30%] [G loss: 1.256551]
2913 [D loss: 0.734556, acc.: 65.23%] [G loss: 1.597937]
2914 [D loss: 0.305575, acc.: 90.23%] [G loss: 2.020714]
2915 [D loss: 0.206874, acc.: 96.09%] [G loss: 2.209316]
2916 [D loss: 0.129148, acc.: 98.44%] [G loss: 2.329169]
2917 [D loss: 0.163974, acc.: 98.05%] [G loss: 2.261389]
2918 [D loss: 0.647810, acc.: 73.44%] [G loss: 1.702506]
2919 [D loss: 0.489521, acc.: 73.05%] [G loss: 1.609058]
2920 [D loss: 0.443176, acc.: 81.64%] [G loss: 1.669490]
2921 [D loss: 0.469312, acc.: 81.64%] [G loss: 1.693962]
2922 [D loss: 0.271479, acc.: 96.09%] [G loss: 1.844363]
2923 [D loss: 0.319873, acc.: 89.06%] [G loss: 1.804127]
2924 [D loss: 0.364035, acc.: 9

3051 [D loss: 0.489090, acc.: 75.00%] [G loss: 1.517210]
3052 [D loss: 1.224192, acc.: 44.53%] [G loss: 1.380574]
3053 [D loss: 0.637778, acc.: 60.16%] [G loss: 1.404071]
3054 [D loss: 0.583340, acc.: 68.36%] [G loss: 1.551685]
3055 [D loss: 0.347413, acc.: 86.72%] [G loss: 1.652291]
3056 [D loss: 0.265299, acc.: 92.58%] [G loss: 1.930435]
3057 [D loss: 0.423322, acc.: 80.47%] [G loss: 1.895259]
3058 [D loss: 0.304434, acc.: 88.28%] [G loss: 2.000517]
3059 [D loss: 0.247479, acc.: 95.31%] [G loss: 2.040079]
3060 [D loss: 0.203665, acc.: 95.31%] [G loss: 2.069944]
3061 [D loss: 0.178073, acc.: 95.70%] [G loss: 2.237636]
3062 [D loss: 0.200387, acc.: 96.48%] [G loss: 2.273567]
3063 [D loss: 2.112958, acc.: 42.58%] [G loss: 1.247322]
3064 [D loss: 1.612800, acc.: 8.59%] [G loss: 0.799617]
3065 [D loss: 0.698611, acc.: 53.52%] [G loss: 0.946527]
3066 [D loss: 0.568533, acc.: 71.48%] [G loss: 1.182729]
3067 [D loss: 1.041058, acc.: 35.16%] [G loss: 1.145637]
3068 [D loss: 0.699921, acc.: 58

3195 [D loss: 0.180590, acc.: 94.92%] [G loss: 1.930092]
3196 [D loss: 0.136445, acc.: 99.22%] [G loss: 2.133468]
3197 [D loss: 0.126313, acc.: 99.61%] [G loss: 2.271779]
3198 [D loss: 0.120847, acc.: 100.00%] [G loss: 2.307345]
3199 [D loss: 0.127731, acc.: 100.00%] [G loss: 2.266805]
3200 [D loss: 0.212605, acc.: 92.58%] [G loss: 2.215295]
3201 [D loss: 0.411681, acc.: 84.38%] [G loss: 1.926452]
3202 [D loss: 0.142882, acc.: 99.22%] [G loss: 1.991359]
3203 [D loss: 0.129695, acc.: 98.44%] [G loss: 2.143569]
3204 [D loss: 0.289356, acc.: 88.67%] [G loss: 2.001130]
3205 [D loss: 0.260692, acc.: 96.88%] [G loss: 1.892322]
3206 [D loss: 1.063503, acc.: 66.02%] [G loss: 1.490815]
3207 [D loss: 1.073359, acc.: 41.02%] [G loss: 1.301851]
3208 [D loss: 1.003249, acc.: 28.12%] [G loss: 1.092794]
3209 [D loss: 0.315053, acc.: 83.20%] [G loss: 1.646437]
3210 [D loss: 0.596824, acc.: 81.25%] [G loss: 1.840359]
3211 [D loss: 1.040448, acc.: 62.50%] [G loss: 1.647602]
3212 [D loss: 0.207329, acc.:

3339 [D loss: 0.444301, acc.: 78.52%] [G loss: 1.868038]
3340 [D loss: 0.377246, acc.: 82.81%] [G loss: 1.864959]
3341 [D loss: 0.483907, acc.: 75.00%] [G loss: 1.866791]
3342 [D loss: 0.537695, acc.: 75.39%] [G loss: 1.604685]
3343 [D loss: 0.715710, acc.: 59.77%] [G loss: 1.418072]
3344 [D loss: 0.447819, acc.: 77.34%] [G loss: 1.598127]
3345 [D loss: 0.526153, acc.: 71.09%] [G loss: 1.598355]
3346 [D loss: 0.583509, acc.: 68.75%] [G loss: 1.502730]
3347 [D loss: 0.485829, acc.: 75.00%] [G loss: 1.542433]
3348 [D loss: 0.493089, acc.: 75.39%] [G loss: 1.715520]
3349 [D loss: 0.614718, acc.: 71.88%] [G loss: 1.567932]
3350 [D loss: 1.167348, acc.: 26.56%] [G loss: 1.233242]
3351 [D loss: 0.785739, acc.: 49.61%] [G loss: 1.406938]
3352 [D loss: 0.506252, acc.: 77.34%] [G loss: 1.525841]
3353 [D loss: 0.883735, acc.: 53.52%] [G loss: 1.427153]
3354 [D loss: 0.978571, acc.: 30.86%] [G loss: 1.331799]
3355 [D loss: 0.468969, acc.: 76.95%] [G loss: 1.422511]
3356 [D loss: 0.528789, acc.: 7

3483 [D loss: 0.300361, acc.: 90.62%] [G loss: 1.606680]
3484 [D loss: 0.230622, acc.: 96.09%] [G loss: 1.736830]
3485 [D loss: 0.777391, acc.: 58.20%] [G loss: 1.355673]
3486 [D loss: 0.542025, acc.: 75.00%] [G loss: 1.213241]
3487 [D loss: 0.376151, acc.: 80.86%] [G loss: 1.360518]
3488 [D loss: 0.360600, acc.: 85.16%] [G loss: 1.529836]
3489 [D loss: 0.356829, acc.: 89.45%] [G loss: 1.592986]
3490 [D loss: 0.338502, acc.: 90.23%] [G loss: 1.570824]
3491 [D loss: 0.454224, acc.: 79.30%] [G loss: 1.473757]
3492 [D loss: 0.327856, acc.: 87.89%] [G loss: 1.535790]
3493 [D loss: 0.277205, acc.: 92.19%] [G loss: 1.751305]
3494 [D loss: 0.628127, acc.: 62.11%] [G loss: 1.509509]
3495 [D loss: 0.621536, acc.: 65.62%] [G loss: 1.412803]
3496 [D loss: 1.846991, acc.: 33.98%] [G loss: 1.117409]
3497 [D loss: 1.195125, acc.: 33.20%] [G loss: 1.068022]
3498 [D loss: 1.051195, acc.: 34.38%] [G loss: 1.130287]
3499 [D loss: 0.545131, acc.: 70.70%] [G loss: 1.423290]
3500 [D loss: 0.384362, acc.: 8

In [ ]:
gan.save_imgs(30000)

In [ ]:
gan.train(5000, batch_size=128, save_interval=1000)

In [ ]:
gan.save_imgs(5000)

In [ ]:
(X_train, _), (_, _) = mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
image_locs = glob("fruits/fruits-360/Training/*/*.jpg")
il = ImageLoader(image_locs, preprocess_input)

In [ ]:
f = il.getNextFiles(10)


In [ ]:
f.shape